# Задание
Собрать информацию о всех строящихся объектах на сайте "наш.дом.рф"
Cохранить результаты в pandas dataframe

Проверить состояние датафрейма и привести его в формат, позволяющий дальнейшее исследование данных: -- Привести колонки датафрейма к одному типу данных -- Проверить индекс, чтобы он соответствовал порядку строк -- Проверить наличие пропусков и в случае обнаружения, определить стратегию их обработки

Провести мини-исследование рынка строящейся недвижимости в одном или нескольких регионах с помощью pandas, matplotlib, seaborn, plotly и других инструментов.

-- Рассчитать средние цены за квадратный метр по регионам. Важно чтобы пропуски не учитывались при расчете средней цены. 

-- Рассчитать объемы вводимого жилья по регионам и по годам. 

-- Сравнить этажность домов, количество квартир, жилую площадь, количество парковочных мест и цену за квадратный метр в разных регионах 

-- Визуализировать разброс, связь и распределение этих показателей по регионам 

-- Сделать выводы по каждому из расчетов и графиков

In [1]:
#Импортирование библиотек
import requests
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
missing_values = ["nan",'N/A']
df = pd.read_csv('our_home_data.csv', na_values = missing_values)
pd.set_option('display.max_columns', None)

# Просмотр DataSet

In [ ]:
df.shape

In [ ]:
df.head(50)

In [ ]:
df.dtypes

In [ ]:
df.describe()

# Чистка

Удаление стобцов, где находится более 50% пустых ячеек

In [3]:
df = df.drop(columns = 'data.developer.bankruptStage.bankruptStageCd')
df = df.drop(columns = 'data.developer.bankruptStage.bankruptStageDesc')
df = df.drop(columns = 'data.developer.bankruptStage.bankruptLawUrl')
df = df.drop(columns = 'data.developer.orgBankruptMsgDttm')
df = df.drop(columns = 'data.metro.id')
df = df.drop(columns = 'data.metro.name')
df = df.drop(columns = 'data.metro.line')
df = df.drop(columns = 'data.metro.color')
df = df.drop(columns = 'data.metro.time')
df = df.drop(columns = 'data.metro.isWalk')
df = df.drop(columns = 'data.metro.colors')
df = df.drop(columns = 'data.transportDistIndexValue')
df = df.drop(columns = 'data.photoRenderDTO')
df = df.drop(columns = 'data.objectTransportInfo')

Очистка строк с NaN из-за невозможности восстановления данных для подзадачи 1

In [4]:
df_avg_P = df.dropna(subset=['data.objPriceAvg','data.developer.regRegionDesc','data.objReady100PercDt'])

Также можно использовать очистку всех строк, где имеются NaN

In [ ]:
df.dropna(axis=0,inplace=True, how='all')

# Анализ Данных

### Рассчитать средние цены за квадратный метр по регионам. Важно чтобы пропуски не учитывались при расчете средней цены.

In [5]:
avg_price_region = df.groupby('data.developer.regRegionDesc').mean()

In [12]:
avg_price_region['data.objPriceAvg'].head(10)

data.developer.regRegionDesc
Адыгея Республика           59190.950000
Алтай Республика            58022.000000
Алтайский край              70012.280488
Амурская область           111318.333333
Архангельская область       86164.833333
Астраханская область        72016.904762
Башкортостан Республика     75488.380208
Белгородская область        78356.300000
Брянская область            58450.020833
Бурятия Республика          58961.035714
Name: data.objPriceAvg, dtype: float64

### Рассчитать объемы вводимого жилья по регионам и по годам

In [13]:
data_prepare = df

In [14]:
c = 0
date_data = []
for i in test['data.objReady100PercDt']:
    if i not in date_data:
        date_data.append(i)

for i in date_data:
    data_prepare = data_prepare.replace(i,i[:4:])

In [15]:
count_data = data_prepare.groupby(['data.developer.regRegionDesc','data.objReady100PercDt']).count()

In [17]:
count_data['data.id']

data.developer.regRegionDesc  data.objReady100PercDt
Адыгея Республика             2022                       8
                              2023                       7
                              2024                       9
                              2025                       6
                              2026                       1
                                                        ..
Ярославская область           2021                       3
                              2022                      26
                              2023                      42
                              2024                      17
                              2025                       5
Name: data.id, Length: 423, dtype: int64

# Визуализация

# Возможно будет нужно

Регион - data.developer.regRegionDesc
Средняя цена по региону - data.objPriceAvg

Восстановление данных для подсчета цены за кв.м в разных регионах не является корректной.


In [ ]:
df['data.objPriceAvg'].fillna(df['data.objPriceAvg'].median(), inplace=True)